Import the required libraries

In [3]:
from d3rlpy.datasets import get_cartpole
from d3rlpy.algos import DiscreteCQL, DQN
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.dataset import Episode
from d3rlpy.ope import DiscreteFQE
from d3rlpy.dataset import MDPDataset

from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

import import_ipynb
import numpy as np
from random import random
from create_dataset import CreateDataset
from FootballEnv import FootballEnv
from tqdm import tqdm

from view import Visualiser

# metrics to evaluate with
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer
from d3rlpy.metrics.scorer import soft_opc_scorer


importing Jupyter notebook from FootballEnv.ipynb


Helper function to create a dummy dataset

In [2]:
def create_dataset():

    dataset_maker = CreateDataset()
    # dataset_maker.loadFile('data.json')
    dataset_maker.loadFilesFromDir('events/*.json')
    observations, actions, rewards = dataset_maker.createEpisodeDataset()
    terminals = np.array([ 0 if (i+1) % dataset_maker.lim == 0 else 1 for i in range(len(actions)) ])
    return MDPDataset(
        observations,
        actions,
        rewards, 
        terminals,
    ), observations

In [ ]:
dataset, observations = create_dataset()
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2, shuffle=False)

In [ ]:
len(train_episodes
)

In [ ]:
s = {}
d = CreateDataset()
for o in observations:
    a = o[-3]
    action = d.ID_to_str[a]

    if action in s: s[action] += 1
    else: s[action] = 1

s

In [ ]:
# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False, batch_size=32,)

# env = FootballEnv(observations)
# env.counter = 0

# start training
output = cql.fit(

    train_episodes,
    # eval_episodes=test_episodes,
    n_epochs=25,
    
    scorers={
        # 'environment': evaluate_on_environment(env), # evaluate with Football Env
        'advantage': discounted_sum_of_advantage_scorer, # smaller is better
        'td_error': td_error_scorer, # smaller is better
        'value_scale': average_value_estimation_scorer # smaller is better
    }
    
)

In [ ]:
output

In [ ]:
# TODO: MAKE THIS WORK!

# off-policy evaluation algorithm
fqe = DiscreteFQE(algo=cql)

# train estimators to evaluate the trained policy
fqe.fit(test_episodes,
   eval_episodes=test_episodes,
   n_epochs=50,
   scorers={
      'init_value': initial_state_value_estimation_scorer,
      'soft_opc': soft_opc_scorer(return_threshold=600)
   }
)

Load Saved Model

In [4]:
path =  "C:\\Users\\micha\\Documents\\Masters\\Football-RL\\d3rlpy_logs\\DiscreteCQL_20220210223510\\"
m2 = DiscreteCQL.from_json(f'{path}params.json')

# ready to load
m2.load_model(f'{path}model_36975.pt')

2022-02-15 20:43.29 [warning  ] Unused arguments are passed.   action_scaler=None algorithm=DiscreteCQL generated_maxlen=100000 real_ratio=1.0


Visualise predictions

In [13]:
ds = CreateDataset()
# ds.loadFilesFromDir('events/*.json')
ds.loadFile('data.json')

# visualiser = Visualiser()

# ds.loadFilesFromDir('events/*.json')

observations, actions, rewards, event_ids = ds.createEpisodeDataset()
answers = {}
players = {}


print(observations.shape, len(event_ids))
for i, situation in tqdm(enumerate(observations)):

    # print(situation)
    predictions = m2.predict([situation])[0]

    p = ds.ID_to_str[predictions]
    if not (p in answers): answers[p] = 1
    else: answers[p] += 1

    # if(p == "shot"):
    #     print(event_ids[i])
    #     print([ds.getPlayerFromActionID(event) for event in event_ids[i]])
    event = ds.getActionFromID(event_ids[i][-2])
    # visualiser.visualise_sequence(situation, 3, predictions, ds.getIDFromAction(event))

    player = event['player']
    if not(player['id'] in players):
        players[player['id']] = {}
        players[player['id']]['name'] = player['name']
        players[player['id']]['good'] = 0
        players[player['id']]['bad'] = 0
        

    # print("Action taken:", ds.ID_to_str[ds.getIDFromAction(event)], p)
    if predictions ==  ds.getIDFromAction(event):
        players[player['id']]['good'] += 1
    else:
        players[player['id']]['bad'] += 1

    if((players[player['id']]['good'] + players[player['id']]['bad']) > 0):
        players[player['id']]['ratio'] = players[player['id']]['good'] / (players[player['id']]['good'] + players[player['id']]['bad']) 
    else:
        players[player['id']]['ratio'] = 0
        
    
    # if(p == "clearance"):
    #     visualiser.visualise_sequence(situation, 3, predictions)

print("finished")

51it [00:00, 509.99it/s]

(281, 15) 281


281it [00:00, 757.40it/s]

finished


In [9]:
len(observations)

295892

In [14]:
len(players)

29

In [15]:
players_2 = {}
for key, player in players.items():
    if 'good' in player and 'bad' in player and (player['good'] + player['bad'] > 5):
        players_2[key] = player


players_sorted =  sorted(players_2.items(), key=lambda x: x[1]['ratio'], reverse=True)
players_sorted[:10]


[(38196,
  {'name': 'Daniel O"Shaughnessy',
   'good': 1,
   'bad': 5,
   'ratio': 0.16666666666666666}),
 (16550,
  {'name': 'Jere Juhani Uronen',
   'good': 1,
   'bad': 10,
   'ratio': 0.09090909090909091}),
 (20845,
  {'name': 'Magomed Ozdoev',
   'good': 1,
   'bad': 12,
   'ratio': 0.07692307692307693}),
 (5179, {'name': 'Daler Kuzyaev', 'good': 1, 'bad': 15, 'ratio': 0.0625}),
 (15416, {'name': 'Jukka Raitala', 'good': 0, 'bad': 8, 'ratio': 0.0}),
 (18735, {'name': 'Dmitriy Barinov', 'good': 0, 'bad': 27, 'ratio': 0.0}),
 (5177, {'name': 'Aleksandr Golovin', 'good': 0, 'bad': 23, 'ratio': 0.0}),
 (16583, {'name': 'Georgiy Dzhikiya', 'good': 0, 'bad': 17, 'ratio': 0.0}),
 (24321, {'name': 'Robin Lod', 'good': 0, 'bad': 9, 'ratio': 0.0}),
 (5171, {'name': 'Roman Zobnin', 'good': 0, 'bad': 14, 'ratio': 0.0})]

In [9]:

players_sorted =  sorted(players_2.items(), key=lambda x: x[1]['ratio'], reverse=True)
players_sorted[:10]


[(23843,
  {'name': 'Diego Mariño Villar',
   'good': 10,
   'bad': 5,
   'ratio': 0.6666666666666666}),
 (24083,
  {'name': 'Manuel Reina Rodríguez',
   'good': 9,
   'bad': 5,
   'ratio': 0.6428571428571429}),
 (26849,
  {'name': 'Júlio César Jacobi',
   'good': 7,
   'bad': 4,
   'ratio': 0.6363636363636364}),
 (6898,
  {'name': 'Jaume Doménech Sánchez', 'good': 9, 'bad': 7, 'ratio': 0.5625}),
 (25870,
  {'name': 'Juan Jesús Calatayud Sánchez',
   'good': 12,
   'bad': 11,
   'ratio': 0.5217391304347826}),
 (6706,
  {'name': 'Francisco Casilla Cortés', 'good': 17, 'bad': 17, 'ratio': 0.5}),
 (5619, {'name': 'Hyeon-Woo Jo', 'good': 7, 'bad': 7, 'ratio': 0.5}),
 (6629,
  {'name': 'Fernando Pacheco Flores',
   'good': 19,
   'bad': 22,
   'ratio': 0.4634146341463415}),
 (25611,
  {'name': 'Vanina Noemí Correa',
   'good': 5,
   'bad': 6,
   'ratio': 0.45454545454545453}),
 (5005,
  {'name': 'Britt Eckerstrom',
   'good': 10,
   'bad': 12,
   'ratio': 0.45454545454545453})]

In [ ]:
counts = {}

for item in actions:
    if not (item in counts): counts[item] = 1
    else: counts[item] += 1

counts


In [ ]:
self.good_events = {
            
            'pass': self.PASS,
            'shot': self.SHOOT,
            'carry':self.CARRY,
            'clearance':self.CLEAR,
            # 'foul won': self.FOUL,
            'foul': self.FOUL,

        }

In [21]:
import json

player_data = {}
events = {}
with open ("data.json", "r") as file:
    events = json.load(file)

ds.ID_to_str[predictions]

count = 0
for event in events:
    if event['type']['name'].lower() in ds.good_events:
        player = event['player']

        if player['id'] in player_data:
        else:
            player_data['id'] = {}
            player_data['id']['name'] = player['name']
            player_data['id']['good'] += 1

count


1971